<a href="https://colab.research.google.com/github/statrav/cnn/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

이번 시간에는 MNIST 데이터셋을 통해 간단한 CNN 모델을 구현해보도록 하겠습니다.

해당 프로젝트를 통해 CNN의 기본 작동 방식을 이해할 수 있을 것입니다. 프로젝트 시작에 앞서, MNIST 데이터셋과 CNN 모델에 대해 간단하게 알아보겠습니다.

### 🎨 MNIST 데이터셋
MNIST는 숫자 0부터 9까지의 손글씨 이미지를 가지고 있는 데이터셋입니다. 총 60,000개의 훈련 이미지와 10,000개의 테스트 이미지로 구성되어 있으며, 각 이미지는 28x28 픽셀 크기의 흑백 숫자 이미지입니다. MNIST 데이터셋은 손글씨 인식 문제를 해결하기 위한 대표적인 예제로 사용되며, 추가로 다양한 기계 학습 알고리즘의 성능을 평가하는 데 자주 활용되는 데이터셋입니다.

### 🧱 CNN
CNN(Convolutional Neural Network)은 주로 이미지 처리 분야에서 사용되는 딥러닝 모델입니다. 여러 개의 컨볼루션 레이어와 풀링 레이어를 사용하여 이미지의 특징을 추출하고, 이를 기반으로 분류 또는 예측 작업을 수행합니다. CNN은 특히 이미지 데이터에서 높은 성능을 발휘하며, 이미지 분류, 객체 탐지, 의료 영상 분석 등 다양한 응용 분야에서 널리 사용됩니다.

## 환경 셋팅

모델 구현을 위해 필요한 모듈과 라이브러리를 적절하게 가져오겠습니다.

In [ ]:
import sys
import tensorflow as tf
import keras
import time
import os

from tensorflow.python.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

## 데이터셋 로드 및 전처리

가장 먼저 `keras.datasets.mnist.load_data()` 함수를 사용하여 MNIST 데이터셋을 로드합니다.

그 다음, 이미지를 28x28 크기로 reshape하고, 픽셀 값을 0~1 범위로 정규화합니다.

마지막으로 레이블을 one-hot 벡터로 변환합니다.

In [ ]:
img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.  # 데이터 정규화
x_test = x_test.astype('float32') / 255.  # 데이터 정규화

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

## 모델 정의

먼저 Sequential 모델을 생성합니다.

그 다음, 두 개의 Conv2D 레이어와 두 개의 MaxPooling2D 레이어를 사용하여 특징을 추출합니다.

이후 Dropout 레이어를 사용하여 과적합을 방지하겠습니다. 또한 Flatten 레이어를 사용하여 2D 특징 맵을 1차원 배열로 변환합니다.

모든 변환이 완료되면, 두 개의 Dense 레이어를 사용하여 분류기를 구성합니다.
첫 번째 Dense 레이어는 1000개의 뉴런을 가지며, 두 번째 Dense 레이어는 클래스 수(10개)만큼의 뉴런을 가지고 소프트맥스 활성화 함수를 사용합니다.

In [ ]:
batch_size = 128
num_classes = 10
epochs = 10

y_train = keras.utils.to_categorical(y_train, num_classes)  # 학습을 위한 원핫벡터로 변경
y_test = keras.utils.to_categorical(y_test, num_classes)  # 원핫벡터로 변경

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())  # fully connected layer
model.add(Dense(1000, activation='relu'))  # 완전연결계층
model.add(Dropout(0.5))  # 과대적합방지를 위해 사용
model.add(Dense(num_classes, activation='softmax'))
model.summary()

## 모델 컴파일 및 학습

이제 모델을 컴파일하고, 학습 데이터셋을 사용하여 모델을 학습시키도록 하겠습니다.

학습 과정 중에는 정확도와 손실 값을 출력함으로써, 학습 과정을 확인할 것입니다.
또한 학습 시간을 참고하기 위해, time 라이브러리를 통해 시간을 저장하겠습니다.

마지막으로 테스트 데이터셋을 사용하여 모델의 최종 성능을 평가합니다.

In [ ]:
start = time.time()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

end = time.time() - start

Test의 손실 값과 정확도를 확인하고, 총 학습 시간을 계산합니다.

모든 과정이 종료되었습니다. 지금까지 CNN 모델을 실제 데이터셋을 활용하여 학습시켜보면서, CNN의 기본 작동 원리에 대해 알아보았습니다.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

print('Training time:', end)